In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
##### for dirname, _, filenames in os.walk('/kaggle/input'):
##### 上一行用于提交Kaggle时使用
for dirname, _, filenames in os.walk('../input/us-patent-phrase-to-phrase-matching'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import torch
from torch import nn
from torch.utils.data import DataLoader

import math
import random
from tqdm import tqdm
from scipy.stats import pearsonr

import numpy as np
import pandas as pd
import os

In [3]:
for dirname, _, filenames in os.walk('./kaggle/input/us-patent-phrase-to-phrase-matching'):
    for filename in filenames:
        if filename == 'train.csv':
            raw_data = pd.read_csv(os.path.join(dirname, filename))
        elif filename == 'test.csv':
            test_data = pd.read_csv(os.path.join(dirname, filename))
        elif filename == 'sample_submission.csv':
            sample_submission = pd.read_csv(os.path.join(dirname, filename))

# Models

### Functions to divide training and testing set

In [4]:
def split_train_eval(data, train_per=0.8, train_num=None):
    if train_num is None:
        train_num = int(len(data) * train_per)
        
    raw_data_index = list(range(len(data)))
    random.shuffle(raw_data_index)
    train_data = raw_data.loc[raw_data_index[:train_num]]
    train_data = train_data.sort_index(ascending=True)
    eval_data = raw_data.loc[raw_data_index[train_num:]]
    eval_data = eval_data.sort_index(ascending=True)
    return train_data, eval_data

train_data, eval_data = split_train_eval(raw_data, train_num=len(raw_data) - 37 * 4)

## Global Paths and Parameters

In [5]:
pretrained_model_path = "./kaggle/working/pretrained_model/"
trained_model_path = "./kaggle/temp/no_context_model/"
save_model_path = "./kaggle/temp/"
batch_size = 64
epoch_num = 6

## Model-fitting I: No Context

Build a SBERT model without considering context.

### Preprocessing of Data

In [6]:
def preprocess_data(data):
    processed_data = []
    for row in data.index:
        inp_example = InputExample(texts=[data.loc[row, 'anchor'],
                                         data.loc[row, 'target']],
                                  label=float(data.loc[row, 'score']))
        processed_data.append(inp_example)
    return processed_data
    
# From https://blog.csdn.net/weixin_44843824/article/details/108257594
def subset(alist, idxs):
    '''
    select the elements with index idx from alist
    '''
    sub_list = []
    for idx in idxs:
        sub_list.append(alist[idx])

    return sub_list

def split_list(alist, group_num=4, shuffle=True, retain_left=False):
    '''
    Divide alist into group_num number of groups.
    shuffle: whether to randomize the order
    retain_left: whether the residual elements become a new group
    '''

    index = list(range(len(alist)))

    if shuffle: 
        random.shuffle(index) 
    
    elem_num = len(alist) // group_num
    sub_lists = {}
    
    for idx in range(group_num):
        start, end = idx*elem_num, (idx+1)*elem_num
        sub_lists['set' + str(idx)] = subset(alist, index[start:end])
    
    if retain_left and group_num * elem_num != len(index):
        sub_lists['set' + str(idx + 1)] = subset(alist, index[end:])
    
    # Return dict and list of values for the selection of the required result
    return sub_lists, list(sub_lists.values())

### Training

The model will be saved for future uses.

In [7]:
def train_model(training_data, validation_data, model_path, batch_size, epoch_num):
    print("Loading model...")
    model_name = "all-mpnet-base-v2"
    model = SentenceTransformer(model_name, device='cuda')
    model.save(model_path)
    # model = SentenceTransformer(model_path, device='cuda')

    train_dataloader = DataLoader(training_data,
                                  shuffle=True,
                                  batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model)
    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_data)
    warmup_steps = math.ceil(len(train_dataloader) * epoch_num * 0.1) # 10% of train data for warm-up
    
    print("Fitting model...")
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=epoch_num,
              evaluator=evaluator,
              warmup_steps=warmup_steps)
    return model


### Output the result

In [8]:
def test_score(model, test_phrases):
    embeddings1 = model.encode(list(test_phrases.loc[:, 'anchor']), convert_to_tensor=True)
    embeddings2 = model.encode(list(test_phrases.loc[:, 'target']), convert_to_tensor=True)
    output_scores = [util.cos_sim(embeddings1[i], embeddings2[i]).item() for i in range(len(test_phrases))]
    
    return output_scores

### Execution of Functions

In [9]:
# Preprocessing
processed_data = preprocess_data(train_data)
_, divided_data = split_list(processed_data, 
                             group_num=5, 
                             shuffle=True,
                             retain_left=False)

# Division of training and validation data
train_processed_data = [x for l in divided_data[:-1] for x in l]
valid_processed_data = divided_data[-1]

# Training
model = train_model(train_processed_data, valid_processed_data, pretrained_model_path, batch_size, epoch_num)
model.save(trained_model_path)  # Save the model to harddisk, preventing memory overflow

# Output the scores
output_scores1 = test_score(model, eval_data)

Loading model...
Fitting model...


C:\Users\Administrator\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/455 [00:00<?, ?it/s]

Iteration:   0%|          | 0/455 [00:00<?, ?it/s]

Iteration:   0%|          | 0/455 [00:00<?, ?it/s]

Iteration:   0%|          | 0/455 [00:00<?, ?it/s]

Iteration:   0%|          | 0/455 [00:00<?, ?it/s]

Iteration:   0%|          | 0/455 [00:00<?, ?it/s]

## Model-fitting II: Devide in Categories (A, B, ...)

### Preprocessing of Data
#### Change the Context to Contain Only the Categories

In [10]:
def simplify_context(data, col_name="context"):
    new_data = data.copy(deep=True)
    for row in new_data.index:
        new_data.loc[row, col_name] = new_data.loc[row, col_name][0]
        
    return new_data

#### Divide by current Context

In [11]:
def divide_context(train_data, col_name="context"):
    all_contexts = train_data.loc[:, col_name].drop_duplicates()
    
    divided_data_list = []
    for context in all_contexts:
        new_data = train_data.loc[train_data.loc[:, col_name] == context]
        divided_data_list.append(new_data)
        
    return divided_data_list, list(all_contexts)

#### Divide Training and Validation set for Every Context

In [12]:
def train_valid_split_by_context(divided_data_list, col_name="context"):
    context_data_split_pairs = {}
    for context_data in divided_data_list:
        context_name = context_data.loc[context_data.index[0], col_name]

        context_processed_data = []
        for row in context_data.index:
            inp_example = InputExample(texts=[context_data.loc[row, 'anchor'],
                                             context_data.loc[row, 'target']],
                                      label=float(context_data.loc[row, 'score']))
            context_processed_data.append(inp_example)
            
        _, divided_context_data = split_list(context_processed_data,
                                             group_num=5,
                                             shuffle=True,
                                             retain_left=False)

        train_context_processed_data = [x for l in divided_context_data[:-1] for x in l]
        valid_context_processed_data = divided_context_data[-1]
        context_data_split_pairs[context_name] = [train_context_processed_data, valid_context_processed_data]
        
    return context_data_split_pairs

### Training Model: Based on Models not Regarding Context

In [13]:
def train_model_by_context(split_divided_data_list, context_list, model_path, save_path, batch_size, epoch_num):
    for context_idx in range(len(context_list)):
        context = context_list[context_idx]
        print("Loading model for %s... (%d / %d)"%(context, context_idx + 1, len(split_divided_data_list.keys())))
        model = SentenceTransformer(model_path)  # Read models not regarding context

        train_dataloader = DataLoader(split_divided_data_list[context][0],
                                      shuffle=True,
                                      batch_size=batch_size)
        train_loss = losses.CosineSimilarityLoss(model)
        evaluator = EmbeddingSimilarityEvaluator.from_input_examples(split_divided_data_list[context][1])
        warmup_steps = math.ceil(len(train_dataloader) * epoch_num * 0.1) # 10% of train data for warm-up

        print("Fitting model for %s... (%d / %d)"%(context, context_idx + 1, len(split_divided_data_list.keys())))
        model.fit(train_objectives=[(train_dataloader, train_loss)],
                  epochs=epoch_num,
                  evaluator=evaluator,
                  warmup_steps=warmup_steps)
        
        model.save(save_path + context)  # Save the model to harddisk, preventing memory overflow

### Output the result

In [14]:
def test_score_context(divided_data_list, all_num, context_list, save_path, col_name="context"):
    output_scores = {}
    with tqdm(total=all_num, desc='evaluate', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        for test_phrases in divided_data_list:
            context = test_phrases.loc[test_phrases.index[0], col_name]
            
            model = SentenceTransformer(save_path + context)
            embeddings1 = model.encode(list(test_phrases.loc[:, 'anchor']), convert_to_tensor=True)
            embeddings2 = model.encode(list(test_phrases.loc[:, 'target']), convert_to_tensor=True)
            
            for i in range(len(test_phrases)):
                output_scores[test_phrases.index[i]] = util.cos_sim(embeddings1[i], embeddings2[i]).item()
            pbar.update(len(test_phrases))
    
    score_list = list(output_scores.items())
    score_list.sort()
    output_scores = [x[1] for x in score_list]
    return output_scores

### Execution of Functions

In [15]:
# Preprocessing
simplified_raw_data = simplify_context(train_data)
simplified_raw_data_list, simplified_all_context = divide_context(simplified_raw_data)

# Division of training and validation data
simplified_split_data_list = train_valid_split_by_context(simplified_raw_data_list)

# Training
train_model_by_context(simplified_split_data_list,
                       simplified_all_context,
                       trained_model_path,
                       save_model_path,
                       batch_size,
                       epoch_num)

# Output the scores
simplified_eval_data = simplify_context(eval_data)
simplified_eval_data_list, simplified_all_eval_context = divide_context(simplified_eval_data)
output_scores2 = test_score_context(simplified_eval_data_list,
                                    len(simplified_eval_data),
                                    simplified_all_eval_context,
                                    save_model_path)

Loading model for A... (1 / 8)
Fitting model for A... (1 / 8)


C:\Users\Administrator\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/75 [00:00<?, ?it/s]

Iteration:   0%|          | 0/75 [00:00<?, ?it/s]

Iteration:   0%|          | 0/75 [00:00<?, ?it/s]

Iteration:   0%|          | 0/75 [00:00<?, ?it/s]

evaluate: 100%|█████████████████████████████████████████████████████| 148/148 [00:05<00:00, 25.5B/s]


## Model-fitting III: Devide in Accurate Context (A01, ...)

### Training Model: Based on Models by Categories

In [16]:
def train_model_by_accurate_context(split_divided_data_list, context_list, model_path, save_path, batch_size, epoch_num):
    for context_idx in range(len(context_list)):
        context = context_list[context_idx]
        print("Loading model for %s... (%d / %d)"%(context, context_idx + 1, len(split_divided_data_list.keys())))
        model = SentenceTransformer(model_path + context[0])  # Read models by categories

        train_dataloader = DataLoader(split_divided_data_list[context][0],
                                      shuffle=True,
                                      batch_size=batch_size)
        train_loss = losses.CosineSimilarityLoss(model)
        evaluator = EmbeddingSimilarityEvaluator.from_input_examples(split_divided_data_list[context][1])
        warmup_steps = math.ceil(len(train_dataloader) * epoch_num * 0.1) # 10% of train data for warm-up

        print("Fitting model for %s... (%d / %d)"%(context, context_idx + 1, len(split_divided_data_list.keys())))
        model.fit(train_objectives=[(train_dataloader, train_loss)],
                  epochs=epoch_num,
                  evaluator=evaluator,
                  warmup_steps=warmup_steps)
        
        model.save(save_path + context)  # Save the model to harddisk, preventing memory overflow

### Execution of Functions

In [17]:
# Preprocessing
raw_data_list, all_context = divide_context(train_data)

# Division of training and validation data
split_data_list = train_valid_split_by_context(raw_data_list)

# Training
train_model_by_accurate_context(split_data_list,
                                all_context,
                                save_model_path,
                                save_model_path,
                                batch_size,
                                epoch_num)

# Output the scores
eval_data_list, all_eval_context = divide_context(eval_data)
output_scores3 = test_score_context(eval_data_list,
                                    len(eval_data),
                                    all_eval_context,
                                    save_model_path)

Loading model for A47... (1 / 106)
Fitting model for A47... (1 / 106)


C:\Users\Administrator\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

evaluate: 100%|█████████████████████████████████████████████████████| 148/148 [06:05<00:00, 2.47s/B]


## 预测数据的进一步处理

### 原始数据

In [18]:
def calc_pearson(arr1, arr2):
    return pearsonr(arr1, arr2)[0]

print(calc_pearson(output_scores1, eval_data['score']))
print(calc_pearson(output_scores2, eval_data['score']))
print(calc_pearson(output_scores3, eval_data['score']))

0.8507180518536596
0.860881129680996
0.8659428886214572


### 方案1：离散化

In [19]:
def to_closest(array):
    interval = [-float('inf'), 0.125, 0.375, 0.625, 0.875, float('inf')]
    new_array = []
    for x in array:
        for i in range(5):
            if x >= interval[i] and x < interval[i + 1]:
                new_array.append(i * 0.25)
                break
    return new_array

new1_score1 = to_closest(output_scores1)
new1_score2 = to_closest(output_scores2)
new1_score3 = to_closest(output_scores3)
print(calc_pearson(new1_score1, eval_data['score']))
print(calc_pearson(new1_score2, eval_data['score']))
print(calc_pearson(new1_score3, eval_data['score']))

0.8338142726035667
0.8297837203327936
0.8368925392227486


### 方案2：投票法，选取落在某区间最多的结果作为离散化后的值

In [20]:
def get_most(arr1, arr2, arr3):
    new_array = []
    available_values = [0, 0.25, 0.5, 0.75, 1]
    for i in range(len(arr1)):
        votes = [0, 0, 0, 0, 0]
        for j in range(5):
            if arr1[i] == available_values[j]:
                votes[j] += 1
            if arr2[i] == available_values[j]:
                votes[j] += 1
            if arr3[i] == available_values[j]:
                votes[j] += 1
                
        # Get the max votes
        max_votes = max(votes)
        if max_votes == 1:  # Three votes are from different values
            # Choose the middle one
            count = 0
            for j in range(5):
                if votes[j] == 1:
                    count += 1
                if count == 1:
                    new_array.append(available_values[j])
                    break
        elif max_votes >= 2:  # At least two votes are from the same values
            new_array.append(available_values[np.argmax(votes)])
            
    return new_array

new2_score = get_most(new1_score1, new1_score2, new1_score3)
print(calc_pearson(new2_score, eval_data['score']))

0.8331244618956068


### 方案3：取距离区间中点最近的结果作为值

In [21]:
def get_closest_distance(arr_list):
    new_array = []
    available_values = [0, 0.25, 0.5, 0.75, 1]
    for i in range(len(arr_list[0])):  # For every element
        min_arr = []
        min_min_distance = float('inf')
        
        for j in range(len(arr_list)):  # Search all columns
            min_distance = float('inf')
            for k in range(5):  # Get the min distance
                min_distance = min(abs(available_values[k] - arr_list[j][i]), min_distance)
        
            if min_distance < min_min_distance:
                min_min_distance = min_distance
                min_arr = [j]
            elif min_distance == min_min_distance:  # Same distance
                min_arr.append(j)
                
        if len(min_arr) > 1:  # Same distance appeared
            all_values = [arr_list[x][i] for x in min_arr]
            value = np.mean(all_values)
        else:
            value = arr_list[min_arr[0]][i]
        
        new_array.append(value)
    return new_array
        
new3_score = get_closest_distance([output_scores1, output_scores2, output_scores3])
print(calc_pearson(new3_score, eval_data['score']))

0.8615199291147393


### 离散化

In [22]:
new4_score = to_closest(new3_score)
print(calc_pearson(new4_score, eval_data['score']))

0.84089544263847


### 方案4：部分离散化（靠近中心的部分进行离散化，其余保持不变）

In [23]:
def partial_to_closest(array, dist):
    interval = [[0 - dist, 0 + dist],
                [0.25 - dist, 0.25 + dist],
                [0.5 - dist, 0.5 + dist],
                [0.75 - dist, 0.75 + dist],
                [1 - dist, 1 + dist]]
    new_array = []
    for x in array:
        flag = False
        for i in range(5):
            if x >= interval[i][0] and x <= interval[i][1]:
                new_array.append(i * 0.25)
                flag = True
                break
        if not flag:
            new_array.append(x)
    return new_array

new5_score = partial_to_closest(output_scores1, 0.03)
print(calc_pearson(output_scores1, eval_data['score']))
print(calc_pearson(new5_score, eval_data['score']))

0.8507180518536596
0.8524250187754796


## 输出

In [24]:
def output_csv(test_data, scores):
    output_data = pd.DataFrame(columns=['id', 'score'])
    output_data['id'] = list(test_data['id'])
    output_data['score'] = list(scores)
    output_data.to_csv('submission.csv', index=False)
    return output_data

output_csv(eval_data, output_scores3)

,id,score
0,6bdd1d05ffa3401e,0.659535
1,d3150b216813544d,0.341476
2,f399a07acd4ae85b,0.320447
3,239628dc0dc62354,0.132301
4,bca3028937ebe118,0.592049
...,...,...
143,104f7725b81c316c,0.476863
144,de732e6b39168f75,0.470938
145,d20e6321d1253bc9,0.142640
146,73a2e15b76aec985,0.083089
